In [27]:
from ultralytics import YOLO
import onnxruntime as ort
from utils import get_image, sigmoid
import torch
import timm
import onnx

In [18]:
img_cls, img_det = get_image("/home/ubuntu/duynd/AIyte/secondnd/input/train/00675cd546313f912cadd4ad54415d69.dicom")

In [19]:
model1 = timm.create_model("tf_efficientnet_b0", num_classes=1)

state_dict = torch.load("/home/ubuntu/ductq/yte/results/ckpt/v5/last.ckpt")["state_dict"]
state_dict = {k.replace('model.', ''):v for k, v in state_dict.items()}

model1.load_state_dict(state_dict)
model1.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm

In [20]:
with torch.no_grad():
    x = torch.tensor(img_cls)
    y = model1(x)

print(sigmoid(y))

tensor([[0.9999]])


In [21]:
onnx_model1 = onnx.load("/home/ubuntu/ductq/yte2/deploy/models/classification_model/1/model.onnx")
ort_sess1 = ort.InferenceSession('/home/ubuntu/ductq/yte2/deploy/models/classification_model/1/model.onnx')
outputs1 = ort_sess1.run(None, {'images': img_cls})
sigmoid(outputs1[0])

array([[    0.99993]], dtype=float32)

In [28]:
model2 = YOLO("/home/ubuntu/ductq/yte2/train/runs/train/weights/last.torchscript")
model2(torch.from_numpy(img_det)/255., device="cuda", conf=0.2, iou=0.6, imgsz=512)
# model.eval()
# model.export(format="onnx",imgsz=512)

# torch.onnx.export(model,                                # model being run
#                   torch.randn(1, 3, 512, 512).cuda(),    # model input (or a tuple for multiple inputs)
#                   "yolo.onnx",           # where to save the model (can be a file or file-like object)
#                   input_names = ['images'],              # the model's input names
#                   output_names = ['output'])    

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /home/ubuntu/ductq/yte2/train/runs/train/weights/last.torchscript for TorchScript inference...

0: 512x512 2 Findings, 9.5ms
Speed: 2.7ms preprocess, 9.5ms inference, 7.4ms postprocess per image at shape (1, 3, 512, 512)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Finding'}
 obb: None
 orig_img: array([[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        ...,
 
        [[ 9,  9,  9],
         [ 9,  9,  9],
         [ 8,  8,  8],
         ...,
         [10, 10, 10],
         [ 9,  9,  9],
         [10, 10, 10]],
 
        [[ 9,  9,  9],
         [ 9,  9,  9],
         [ 8,  8,  8],
         ...,
         [10, 10, 10],
         [10, 10, 10],
         [10, 10, 10]],
 
        [[10, 10, 10],
   

In [24]:
import onnxruntime as ort

# Create a session with CUDAExecutionProvider
ort_sess2 = ort.InferenceSession('/home/ubuntu/ductq/yte2/deploy/models/detection_model_1/1/model.onnx')
outputs2 = ort_sess2.run(None, {'images': img_det})

# Check the active execution providers
# print("Active providers:", sess.get_providers())

In [25]:
import numpy as np

In [14]:
img_det.shape

(1, 3, 512, 512)

In [47]:
yolo(torch.tensor(img_det),conf=0.5)


WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 255.0. Dividing input by 255.
0: 512x512 2 Findings, 37.4ms
Speed: 1.1ms preprocess, 37.4ms inference, 18.1ms postprocess per image at shape (1, 3, 512, 512)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Finding'}
 obb: None
 orig_img: array([[[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ...,
         [ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0]],
 
        ...,
 
        [[ 9,  9,  9],
         [ 9,  9,  9],
         [ 8,  8,  8],
         ...,
         [10, 10, 10],
         [ 9,  9,  9],
         [10, 10, 10]],
 
        [[ 9,  9,  9],
         [ 9,  9,  9],
         [ 8,  8,  8],
         ...,
         [10, 10, 10],
         [10, 10, 10],
         [10, 10, 10]],
 
        [[10, 10, 10],
   

In [29]:
import torch

In [42]:
model_ts = torch.jit.load("/home/ubuntu/ductq/yte2/train/runs/train/weights/last.torchscript", _extra_files={"config.txt": ""})